In [2]:
import os 
import torch 
import tqdm 
import numpy as np 
import pandas as pd 

from torch import nn
from torchvision import models
from torchvision import transforms

from torch.optim.lr_scheduler import CosineAnnealingLR

import torch.nn.functional as F 

from dataformm import TrainDataset, TestDataset

In [17]:
BATCH_SIZE = 64
EPOCHS = 10

SEED = 31

In [11]:
train_loader = TrainDataset()
test_loader = TestDataset()
train_loader.landsat = np.moveaxis(train_loader.landsat_data, 2, 1)
train_loader.landsat.shape

(88468, 6, 60)

In [23]:
class MultimodalEnsemble(nn.Module):
    def __init__(self, num_classes) -> None:
        super(MultimodalEnsemble, self).__init__()

        self.landsat_norm = nn.LayerNorm([60, 6])
        self.landsat_model = models.resnet18(weights=None)
        self.landsat_model.conv1 = nn.Conv2d(6, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.landsat_model.maxpool = nn.Identity()

        self.pca_norm = nn.LayerNorm([1, 59])
        self.pca_model = models.resnet18(weights=None)
        self.pca_model.conv1 = nn.Conv2d(1, 64, kernel_size=1, stride=1, padding=1, bias=False)
        self.pca_model.maxpool = nn.Identity()

        self.sentinel_model = models.swin_t(weights="IMAGENET1K_V1")
        self.sentinel_model.features[0][0] = nn.Conv2d(4, 96, kernel_size=(4, 4), stride=(4, 4))
        self.sentinel_model.head = nn.Identity()

        self.ln1 = nn.LayerNorm(1000)
        self.ln2 = nn.LayerNorm(1000)
        self.fc1 = nn.Linear(2768, 4096)
        self.fc2 = nn.Linear(4096, num_classes)

        self.dropout = nn.Dropout(p=0.3)

    def forward(self, x, y, z):
        x = self.landsat_norm(x)
        x = self.landsat_model(x)
        x = self.ln1(x)

        y = self.pca_norm(y)
        y = self.pca_model(y)
        y = self.ln2(y)

        z = self.sentinel_model(z)

        xyz = torch.cat((x, y, z), dim=1)
        xyz = self.fc1(xyz)
        xyz = self.dropout(xyz)
        return self.fc2(xyz)


In [19]:
def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True 
        torch.backends.cudnn.benchmark = False 

set_seed(SEED)

In [21]:
device = torch.device("cpu")

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("[DEVICE] CUDA")

num_classes = train_loader.labels.shape[1]

[DEVICE] CUDA


In [24]:
model = MultimodalEnsemble(num_classes=num_classes).to(device=device)

AttributeError: 'MultimodalEnsemble' object has no attribute 'sentinel_model_features'